In [1]:
import pandas as pd
import jutil
from my_fp_growth import JFPGrowth

In [2]:
df = jutil.load_pd_df("select * from view_stu_course_mark where course_type='必' and speciality_code='0101' and student_id like '2013%'")
df.head()

,college_code,course_code,course_name,course_type,grade,mark,pmark,speciality_code,speciality_name,student_id,term_order
0,01,0153830,土木工程地质,必,2013,6,75.0,0101,土木工程,20130002,3
1,01,1111310,线性代数M,必,2013,0,38.0,0101,土木工程,20130002,2
2,01,1118903,大学物理实验MⅡ,必,2013,5,74.0,0101,土木工程,20130002,2
3,01,1571045,计算机程序设计与数据库基础M,必,2013,4,68.0,0101,土木工程,20130002,1
4,01,3000300,电工与电子技术基础B,必,2013,7,79.0,0101,土木工程,20130002,3


In [3]:
data = df[~df.course_name.str.startswith("形势与政策")]
data = data[~data.course_name.str.startswith("军事技能")]
data = data[~data.course_name.str.startswith("土木工程认识实习")]
data.count()

college_code       30410
course_code        30410
course_name        30410
course_type        30410
grade              30410
mark               30410
pmark              30410
speciality_code    30410
speciality_name    30410
student_id         30410
term_order         30410
dtype: int64

In [4]:
course_df = df.groupby('course_code')

In [5]:
pmark_mean = course_df.agg({'pmark':'mean'})
course_mark_mean_dict = pmark_mean.to_dict()['pmark'] # 得到课程和其平均分的dict

In [6]:
dg = data.groupby('student_id').groups

input_list = []
my_input = []
out = data['college_code'].count()
try:
    for k in dg:
        ids = dg[k]
        items = []
        for id in ids:
            if id >= out : continue
            course_code = data.iloc[id].get('course_code')
            pmark = data.iloc[id].get('pmark')
            mark = 0
            if pmark > course_mark_mean_dict[course_code]: mark = 1
            items.append(course_code+'_'+str(mark))
        items = list(set(items)) # 去重
        input_list.append((k,items))
        my_input.append(items)
except Exception as e:
    print(e)
len(input_list)

535

In [27]:
from pyspark.ml.fpm import FPGrowth

ddd = spark.createDataFrame(input_list,['id','items'])

fpG = FPGrowth(itemsCol="items",minSupport=0.05,minConfidence=0.6)
model = fpG.fit(ddd)

In [28]:
model.freqItemsets.show(n=10,truncate=False)

+---------------------------------+----+
|items                            |freq|
+---------------------------------+----+
|[6111120_0]                      |140 |
|[6111120_0, 0153816_0]           |50  |
|[6111120_0, 0153816_0, 9991999_0]|32  |
|[6111120_0, 0153816_0, 6011320_1]|27  |
|[6111120_0, 0153816_0, 6321910_0]|27  |
|[6111120_0, 0153816_0, 6321920_1]|27  |
|[6111120_0, 0153816_0, 6321910_1]|37  |
|[6111120_0, 0153570_1]           |58  |
|[6111120_0, 0153570_1, 6111110_1]|28  |
|[6111120_0, 0153570_1, 9991999_0]|37  |
+---------------------------------+----+
only showing top 10 rows



In [29]:
pdc = model.associationRules.toPandas()

In [38]:
pdc.count()

antecedent    68992
consequent    68992
confidence    68992
dtype: int64

In [31]:
# 提取出课程代码和课程名称的对应
course_arr = data.loc[:,['course_code','course_name']].values
course_dict = {}
for row in course_arr:
    course_dict[row[0]] = row[1]
len(course_dict)

antecedent = pdc['antecedent']
consequent = pdc['consequent']

consequent_name = [course_dict.get(row[0][:row[0].index('_')],'unknown') for row in consequent ]

antecedent_name = [[course_dict.get(code[:code.index('_')],'unknown') for code in row] for row in antecedent]
# for row in antecedent:
#     antecedent_name.append([course_dict.get(code[:code.index('_')],'unknown') for code in row])

In [32]:
for i in range(40):
    print('%s-->%s'%(antecedent_name[i],consequent_name[i]))

['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->土力学B
['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->大学物理AⅠ
['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->课外创新实践
['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->土木工程概论（认识实习）
['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->理论力学AⅡ
['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->理论力学AⅠ
['体育Ⅳ', '材料力学AⅠ', '高等数学Ⅱ']-->大学物理AⅡ
['军事理论', '土力学B', '大学物理AⅠ']-->结构力学AⅠ
['计算机绘图实习', '大学物理实验Ⅰ', '理论力学AⅡ']-->高等数学Ⅱ
['计算机绘图实习', '大学物理实验Ⅰ', '理论力学AⅡ']-->土木工程概论（认识实习）
['土木工程地质实习', '土木工程概论（认识实习）', '理论力学AⅠ']-->理论力学AⅡ
['工程测量实习', '大学物理AⅡ', '理论力学AⅡ', '理论力学AⅠ']-->高等数学Ⅱ
['工程化学', '大学物理AⅠ', '理论力学AⅠ']-->课外创新实践
['工程化学', '大学物理AⅠ', '理论力学AⅠ']-->理论力学AⅡ
['工程化学', '大学物理AⅠ', '理论力学AⅠ']-->材料力学AⅠ
['体育Ⅳ', '土力学B', '课外创新实践']-->理论力学AⅠ
['体育Ⅳ', '土力学B', '课外创新实践']-->高等数学Ⅱ
['体育Ⅳ', '土力学B', '课外创新实践']-->理论力学AⅡ
['计算机绘图实习', '线性代数B', '材料力学AⅡ']-->理论力学AⅠ
['英语Ⅰ', '英语Ⅳ', '土力学B']-->结构力学AⅠ
['英语Ⅰ', '英语Ⅳ', '土力学B']-->材料力学AⅡ
['英语Ⅰ', '英语Ⅳ', '土力学B']-->课外创新实践
['英语Ⅰ', '英语Ⅳ', '土力学B']-->高等数学Ⅱ
['英语Ⅰ', '英语Ⅳ', '土力学B']-->结构力学BⅡ
['英语Ⅰ', '英语Ⅳ', '土力学B']-->土木工程概论（认识实习）
['英语Ⅰ', '英语Ⅳ', '土力学B']-->土木工程制图III-A
['马克思主义基本原理', '结构力学BⅡ']-->土木工程

In [18]:
mfp = JFPGrowth(my_input,minSupport=0.1,minConfidence=0.6)

In [19]:
i = mfp.getFreqItems()

In [20]:
my_rules = mfp.getFinalRules()
len(my_rules)

2498

In [21]:
my_rules[:10]

[(frozenset({'0153821_1'}), frozenset({'6321910_1'}), 0.6511627906976745),
 (frozenset({'0171040_1'}), frozenset({'9992056_0'}), 0.625),
 (frozenset({'0171040_1'}), frozenset({'6321910_1'}), 0.65625),
 (frozenset({'0153819_0'}), frozenset({'9992056_0'}), 0.6019417475728155),
 (frozenset({'0121000_0'}), frozenset({'9991999_0'}), 0.6017699115044248),
 (frozenset({'0121000_0'}), frozenset({'6321910_1'}), 0.6194690265486725),
 (frozenset({'6111120_0'}), frozenset({'6321910_1'}), 0.6029411764705882),
 (frozenset({'0121000_1', '9992056_0'}),
  frozenset({'9991999_0'}),
  0.6962025316455697),
 (frozenset({'0121000_1', '9991999_0'}),
  frozenset({'9992056_0'}),
  0.6179775280898876),
 (frozenset({'0121000_1'}), frozenset({'9991999_0'}), 0.644927536231884)]